In [12]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score,matthews_corrcoef,accuracy_score, confusion_matrix, recall_score, roc_auc_score, precision_score, f1_score
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
import random
import math
from datetime import timedelta
import pickle
import datetime
from datetime import datetime
from yahoo_earnings_calendar import YahooEarningsCalendar
import math

In [13]:
with open('updated_sml_raw_tickers.dictionary', 'rb') as config_dictionary_file:
 
    # Step 3
    combined = pickle.load(config_dictionary_file)

In [14]:
b4rep_window = 10
thr = .65


In [15]:
def compress(series, sides=0.01):
    
    series = pd.Series(np.where( series > series.quantile(q=(1-sides)) , series.quantile(q=(1-sides)), series ), 
                      index=series.index)
    series = np.where( series < series.quantile(q=sides), series.quantile(q=sides), series)
    return series

def compress_right_tail(series, sides=0.01):
    
    series = pd.Series(np.where( series > series.quantile(q=(1-sides)) , series.quantile(q=(1-sides)), series ), 
                      index=series.index)
    return series

def create_features(dtf, tick, lead=5, ret_lag=[1,2,5,10,20,44,261], tr=0.025, vol_lag=[1,2,5,10,20,44,261], vol_window=100, sides=0.01):
    
    feat_cols = []
#    dtf['volume'] = np.log(dtf['volume'])

#    dtf['volume'] = (dtf['volume']-dtf['volume'].cummin())/(dtf['volume'].cummax()-dtf['volume'].cummin())

    avg_vol_name = 'avg_vol_' + str(vol_window)
    dtf[avg_vol_name] = dtf['volume'].rolling(window=vol_window).mean()
    dtf[avg_vol_name] = np.log(dtf[avg_vol_name])
#     avg_range_name = 'avg_range_' + str(vol_window)
#     dtf[avg_range_name] = np.abs(dtf['high'].rolling(window=vol_window).mean()-dtf['low'].rolling(window=vol_window).mean())
#     dtf[avg_range_name] = compress(dtf[avg_range_name], sides=sides)

# #     avg_low_name = 'avg_low_' + str(vol_window)
# #     dtf[avg_low_name] = dtf[tick]-dtf['low'].rolling(window=vol_window).mean()
# #     avg_high_name = 'avg_high_' + str(vol_window)
# #     dtf[avg_high_name] = dtf['high'].rolling(window=vol_window).mean()-dtf[tick]
    
#     for avg in vol_avgs:

#         name_simple = 'open'+str(avg)
#         dtf[name_simple] = dtf['open'].rolling(window=avg).mean()
#         dtf[name_simple] = np.log(dtf[name_simple])
#         dtf[name_simple] = compress(dtf[name_simple], sides=sides)
#         dtf[name_simple] = (dtf[name_simple]-dtf[name_simple].min()) / (dtf[name_simple].max() - dtf[name_simple].min())
#         feat_cols.append(name_simple)
#         name_simple = 'high'+str(avg)
#         dtf[name_simple] = dtf['high'].rolling(window=avg).mean()
#         dtf[name_simple] = np.log(dtf[name_simple])
#         dtf[name_simple] = compress(dtf[name_simple], sides=sides)
#         dtf[name_simple] = (dtf[name_simple]-dtf[name_simple].min()) / (dtf[name_simple].max() - dtf[name_simple].min())
#         #feat_cols.append(name_simple)
#         name_simple = 'low'+str(avg)
#         dtf[name_simple] = dtf['low'].rolling(window=avg).mean()
#         dtf[name_simple] = np.log(dtf[name_simple])
#         dtf[name_simple] = compress(dtf[name_simple], sides=sides)
#         dtf[name_simple] = (dtf[name_simple]-dtf[name_simple].min()) / (dtf[name_simple].max() - dtf[name_simple].min())
#         #feat_cols.append(name_simple)
# #         name_simple = 'vwap'+str(avg)
# #         dtf[name_simple] = dtf['vwap'].rolling(window=avg).mean()
# #         dtf[name_simple] = np.log(dtf[name_simple])
# #         dtf[name_simple] = compress(dtf[name_simple], sides=sides)
# #         dtf[name_simple] = (dtf[name_simple]-dtf[name_simple].min()) / (dtf[name_simple].max() - dtf[name_simple].min())
# #         feat_cols.append(name_simple)
#         name_simple = 'ranges'+str(avg)
#         dtf[name_simple] = np.abs(dtf['high'+str(avg)]-dtf['low'+str(avg)])
#         dtf[name_simple] = compress(dtf[name_simple], sides=sides)
#         dtf[name_simple] = dtf[name_simple] / dtf[avg_range_name]
#         dtf[name_simple] = compress(dtf[name_simple], sides=sides)
#         #feat_cols.append(name_simple)


    
    
    for lag in vol_lag:
        name_simple = 'vol'+str(lag)
        dtf[name_simple] = dtf['volume'].rolling(window=lag).mean()
        dtf[name_simple] = np.log(dtf[name_simple])
        dtf[name_simple] = dtf[name_simple] / dtf[avg_vol_name]
        dtf[name_simple] = compress(dtf[name_simple], sides=sides)
        dtf[name_simple] = (dtf[name_simple]-dtf[name_simple].cummin()) / (dtf[name_simple].cummax() - dtf[name_simple].cummin())
        feat_cols.append(name_simple)
    for lag in ret_lag:
        name = 'ret'+str(lag)
        dtf[name] = dtf[tick]/dtf[tick].shift(lag)-1
        dtf[name] = compress(dtf[name], sides=sides)
        dtf[name] = (dtf[name] - dtf[name].cummin()) / (dtf[name].cummax() - dtf[name].cummin())
        feat_cols.append(name)

        
        
    dtf['tick'] = tick
    #feat_cols.append(tick)
#    dtf['fwd'] = dtf[tick].shift(-lead+1) / dtf[tick]-1
#    dtf['fwd'] = dtf[tick].shift(-lead) / dtf[tick]-1
    dtf['fwd'] = dtf[tick].shift(-lead-1) / dtf[tick]-1
    dtf['ycol'] = np.where(dtf['fwd'] >= tr, 1, 0)
    feat_cols.append('ycol')
#    dtf['mva_90'] = dtf[tick].rolling(window=90).mean()
#    dtf['y_90'] = np.where(dtf[tick] >= dtf['mva_90'], 1, 0)
#    feat_cols.append('y_90')
    return dtf, feat_cols

In [16]:
def ticker_features(combined_dtfs, ticker, growth_tr=0.02, back_window=28, b4rep_window=10,
                   ret_lag=[1,2,5,10,20,44,261], vol_lag=[1,2,5,10,20,44,261], vol_window=100, sides=0.01):

    idf = combined_dtfs[ticker]
    if combined[ticker].shape[0]==0:
        return
    else:
        idf['idate'] = idf.index
        idf['idate'] = idf['idate'].shift(1)
        idf['b4rep_window'] = idf.index
        idf['b4rep_window'] = idf['b4rep_window'].shift(b4rep_window+1)
        #print(ticker)
        idf['back_window'] = pd.to_datetime(np.where(idf.rep_date == 1, idf['idate'] - \
                                                     pd.Timedelta(days=back_window), pd.to_datetime(np.nan)))
        idf['back_window'] = idf['back_window'].fillna(method='backfill')

        idf['b4rep_window'] = pd.to_datetime(np.where(idf.rep_date == 1, idf['b4rep_window'], pd.to_datetime(np.nan)))
        idf['b4rep_window'] = idf['b4rep_window'].fillna(method='backfill')

        idf['b4rep'] = np.where( idf.idate >= idf.back_window , 1, 0 )
        idf['next_rep'] = pd.to_datetime(np.where( idf.rep_date == 1, idf.idate, pd.to_datetime(np.nan)))
        idf['next_rep'] = idf['next_rep'].fillna(method='backfill')
        idf['price_atrep'] = idf['next_rep'].map(idf[ticker])
        idf['price_b4rep'] = idf['b4rep_window'].map(idf[ticker])
        idf['b4rep_chng'] = idf['price_atrep']/idf['price_b4rep']-1

        #idf = idf[idf.b4rep == 1]#[['meli', 'volume', 'rep_date']]
        #idf['ycol'] = np.where( (idf['b4rep'] == 1) & (idf['b4rep_chng'] > growth_tr), 1, 0 )
        #idf = idf[[ticker, 'volume', 'b4rep', 'b4rep_chng']]

        #idf = create_returns(idf, ticker, lead=b4rep_window, tr=growth_tr)
        idf, feat_cols = create_features(idf, ticker, lead=b4rep_window,ret_lag=ret_lag, vol_lag=vol_lag, tr=growth_tr, 
                            vol_window=vol_window, sides=sides)
        feat_cols.append('fwd')
        feat_cols.append('tick')
        idf = idf[idf.idate == idf.b4rep_window]

        return idf[feat_cols].dropna()

In [17]:
def nasdaq_features(combined, params=None):
    
    if params == None:
        params = {
            'b4rep_window': b4rep_window, 
            'growth_tr': 0.02,
            'back_window': 28, 
            'ret_lag': [1,2,5,10,22,44,51,66,118,132,246,261,375,480,520], 
            'vol_lag': [1,2,5,10,22,44,51,66,118,132], 

            'vol_window': 100, 
            'sides': 0.01
        }
    
    df = pd.DataFrame()
    for i in combined:
        df = df.append(ticker_features(combined, i, **params))
    df = df.sort_index()
    
    return df

In [18]:
%%time
df = nasdaq_features(combined, params=None)
#### ADDED COLUMN FOR TESTING RETURNS
#df['case_id'] = np.linspace(1, df.shape[0], df.shape[0])
df.index = df.index
df['date:tick'] = df.index.astype(str)+':'+df['tick']

#### ALSO AMENDED my_tss function to account for new column
df.head()

CPU times: user 53.2 s, sys: 0 ns, total: 53.2 s
Wall time: 53.3 s


,vol1,vol2,vol5,vol10,vol22,vol44,vol51,vol66,vol118,vol132,...,ret132,ret246,ret261,ret375,ret480,ret520,ycol,fwd,tick,date:tick
2002-02-01,0.572067,0.477302,0.546161,0.411387,0.298953,0.169635,0.163899,0.143142,0.940024,0.924398,...,0.515827,0.321486,0.529665,0.936267,0.900037,1.000000,0,0.007042,npk,2002-02-01:npk
2002-02-04,0.522283,0.451851,0.378945,0.423369,0.530137,0.426066,0.482181,0.545998,0.421985,0.512921,...,0.285365,0.057355,0.000000,0.236055,0.184089,0.047927,0,0.018896,sm,2002-02-04:sm
2002-02-04,0.356284,0.377492,0.359212,0.370669,0.360424,0.350592,0.322257,0.350607,0.681653,0.728864,...,0.457348,0.297932,0.337141,0.448321,0.895728,0.000000,0,-0.036990,awr,2002-02-04:awr
2002-02-05,0.735692,0.632262,0.569987,0.501869,0.451612,0.421432,0.443781,0.468898,0.767738,0.867283,...,0.168679,0.042063,0.006122,0.099430,0.576552,0.790780,0,-0.154000,tile,2002-02-05:tile
2002-02-05,0.349647,0.437226,0.408945,0.458498,0.262206,0.246841,0.282244,0.376168,0.612601,0.268417,...,0.569583,0.767060,0.608266,0.795958,0.229360,1.000000,0,-0.246296,twi,2002-02-05:twi


In [19]:
# save the model to disk
filename = 'inference_before_sml.sav'
 
# some time later...
 
# load the model from disk
estimatorCopy = pickle.load(open(filename, 'rb'))

In [20]:
cols=['vol1', 'vol2', 'vol5', 'vol10', 'vol22', 'vol44', 'vol51', 'vol66',
       'vol118', 'vol132', 'ret1', 'ret2', 'ret5', 'ret10', 'ret22', 'ret44',
       'ret51', 'ret66', 'ret118', 'ret132', 'ret246', 'ret261', 'ret375',
       'ret480', 'ret520']
num_companies=600
next_reports = df['tick'][-int(num_companies*0.5):]
ticker_proba = {}
existing_reports = []
for ticker in next_reports:
    try:
        pred_features=combined[ticker][-1:][cols]
        ticker_proba[ticker] = estimatorCopy.predict_proba(pred_features)[:,2]
        existing_reports.append(ticker)
    except:
        print('except:')
        print(ticker)

In [21]:
{k: v for k, v in sorted(ticker_proba.items(), key=lambda item: item[1],reverse=True)}

{'sji': array([0.76120401]),
 'ufcs': array([0.71616499]),
 'nwn': array([0.68964803]),
 'psb': array([0.67027393]),
 'ingn': array([0.66396719]),
 'll': array([0.6634257]),
 'hpr': array([0.66108457]),
 'stba': array([0.65585284]),
 'sr': array([0.64510272]),
 'crvl': array([0.63987896]),
 'sm': array([0.63943303]),
 'iivi': array([0.63167702]),
 'vvi': array([0.62543399]),
 'lmnx': array([0.61888039]),
 'tti': array([0.61047938]),
 'acor': array([0.60613951]),
 'ait': array([0.60417264]),
 'nmih': array([0.59781812]),
 'bfs': array([0.58831024]),
 'ipar': array([0.58483039]),
 'akr': array([0.56844243]),
 'wd': array([0.56335404]),
 'plce': array([0.55858417]),
 'bcor': array([0.55792324]),
 'ahh': array([0.54996815]),
 'cal': array([0.54824813]),
 'kop': array([0.54585921]),
 'hsc': array([0.54448957]),
 'fsp': array([0.54303233]),
 'rpt': array([0.54276159]),
 'inn': array([0.54043638]),
 'pgti': array([0.53973563]),
 'tisi': array([0.53697245]),
 'cps': array([0.53174869]),
 'cbl'

In [22]:
import datetime
from datetime import datetime
from yahoo_earnings_calendar import YahooEarningsCalendar
import math

my_custom_delay_s = 0.01

yec = YahooEarningsCalendar(my_custom_delay_s)

# Returns the next earnings date of BOX in Unix timestamp
#print(yec.get_next_earnings_date('box'))
# 1508716800

ticker_date = {}
sml_tickers = []
today=datetime.today()
start=today+timedelta(days=14)
#end=today+timedelta(days=16)
end=today+timedelta(days=18)

for ticker in set(next_reports):
    pred_features=combined[ticker][-1:][cols]
    prediction = estimatorCopy.predict_proba(pred_features)[:,1]
    index = str(ticker)+':'+str(math.floor(100*prediction))

    try:
        date = datetime.fromtimestamp(yec.get_next_earnings_date(ticker))
        ticker_date[index] = date
        if date>start and date<end:
            sml_tickers.append(ticker)
    except:
        ticker_date[index] = datetime(3000, 1, 1, 1, 1)

{k: v for k, v in sorted(ticker_date.items(), key=lambda item: item[1])}


{'cts:31': datetime.datetime(2020, 10, 29, 6, 0),
 'patk:22': datetime.datetime(2020, 10, 29, 6, 0),
 'mye:18': datetime.datetime(2020, 10, 29, 6, 0),
 'belfb:22': datetime.datetime(2020, 10, 30, 6, 0),
 'sup:20': datetime.datetime(2020, 11, 2, 6, 0),
 'gifi:23': datetime.datetime(2020, 11, 2, 6, 0),
 'egrx:37': datetime.datetime(2020, 11, 2, 6, 0),
 'dea:26': datetime.datetime(2020, 11, 2, 6, 0),
 'tcmd:26': datetime.datetime(2020, 11, 2, 6, 0),
 'kopn:24': datetime.datetime(2020, 11, 3, 6, 0),
 'sppi:20': datetime.datetime(2020, 11, 4, 6, 0),
 'ingn:10': datetime.datetime(2020, 11, 4, 6, 0),
 'srdx:36': datetime.datetime(2020, 11, 4, 6, 0),
 'dfin:22': datetime.datetime(2020, 11, 4, 6, 0),
 'vtol:31': datetime.datetime(2020, 11, 4, 6, 0),
 'anik:41': datetime.datetime(2020, 11, 4, 6, 0),
 'uihc:9': datetime.datetime(2020, 11, 5, 6, 0),
 'anip:11': datetime.datetime(2020, 11, 5, 6, 0),
 'ueic:28': datetime.datetime(2020, 11, 5, 6, 0),
 'pgti:13': datetime.datetime(2020, 11, 5, 6, 0),


In [23]:
df = pd.DataFrame(sml_tickers)
writer = pd.ExcelWriter('sml_tickers.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='welcome', index=False)
writer.save()

In [24]:
len(df)

19

In [25]:
df

,0
0,ahh
1,nsit
2,cry
3,akr
4,veco
5,cohu
6,ande
7,vvi
8,krg
9,acls
